In [63]:
import numpy as np
import copy

Parameters and function setting

In [64]:
def solution(x, y):
    return np.sin(3.0 * np.pi * x ** 2 * y) + (y - 1.0) ** 3

In [65]:
def f(x, y):
    return 18 * np.pi * (np.pi * x ** 2 * np.sin(3 * np.pi * x ** 2 * y) * (x ** 2 + 6 * y ** 2) - y * np.cos(3 * np.pi * x ** 2 * y)) - 12 * (y - 1)

In [66]:
def a(x, y):
    return 3

def b(x, y):
    return 2

Mapping function

In [67]:
def idx(i, j, N):
    return (j-1) * (N - 1) + (i-1)

SLAE compilation

In [68]:
def SLAE_compilation(N: int):
    h = 1.0 / N
    size = (N - 1) ** 2 
    
    A = np.zeros((size, size))
    rhs = np.zeros(size)
    
    coeff_x = a(0, 0) / h**2
    coeff_y = b(0, 0) / h**2
    
    for j in range(1, N):  # y
        for i in range(1, N):  # x
            k = idx(i, j, N=N)
            x, y = i*h, j*h
            
            # diagonal
            A[k, k] = 2*(a(0, 0)+b(0, 0))/h**2
            rhs[k] = f(x, y) 
            
            # neighbours
            # east (i+1,j)
            if i < N-1:
                A[k, idx(i+1,j, N)] = -coeff_x
            else:  # border x=1
                rhs[k] += coeff_x * solution(1.0, y)
            
            # west (i-1,j)
            if i > 1:
                A[k, idx(i-1,j, N)] = -coeff_x
            else:  # border x=0
                rhs[k] += coeff_x * solution(0.0, y)
            
            # nord (i,j+1)
            if j < N-1:
                A[k, idx(i,j+1, N)] = -coeff_y
            else:  # border y=1
                rhs[k] += coeff_y * solution(x, 1.0)
            
            # sud (i,j-1)
            if j > 1:
                A[k, idx(i,j-1, N)] = -coeff_y
            else:  # border y=0
                rhs[k] += coeff_y * solution(x, 0.0)
    
    return A, rhs

In [69]:
def exact_vector(N):
    h = 1.0 / (N - 1)
    u_ex = np.zeros((N-1)**2)
    for j in range(1, N):
        for i in range(1, N):
            k = idx(i,j, N=N)
            u_ex[k] = solution(i*h, j*h)

    return u_ex

In [70]:
N = 5
A, b_vec = SLAE_compilation(N)

In [71]:
u_ex = exact_vector(N)

Conjugate gradient method

In [72]:
def conjugate_gradient(A: np.ndarray, b_vec: np.ndarray, eps: float = 1e-8) -> np.ndarray:
    x_cur = np.zeros_like(b_vec)
    r_cur = b_vec - A @ x_cur
    p_cur = copy.deepcopy(r_cur)

    while np.linalg.norm(r_cur) > eps:
        r_prev = copy.deepcopy(r_cur)
        p_prev = copy.deepcopy(p_cur)
        alpha_prev = np.dot(r_prev, r_prev) / np.dot(A @ p_prev, p_prev)
        x_prev = copy.deepcopy(x_cur)
        x_cur = x_prev + alpha_prev * p_prev
        r_cur = r_prev - alpha_prev * A @ p_prev
        beta_prev = np.dot(r_cur, r_cur) / np.dot(r_prev, r_prev)
        p_cur = r_cur + beta_prev * p_prev
        # print(f"norm(r_cur) = {np.linalg.norm(r_cur)}\n")
    return x_cur

In [ ]:
u_cg = conjugate_gradient(A, b_vec)
max(A @ u_cg - b_vec)

Richardson method

In [79]:
def Richardson(A: np.ndarray, b_vec: np.ndarray, eps: float = 1e-8) -> np.ndarray:

    eigvals = np.linalg.eigvals(A)
    lambda_max = max(np.abs(eigvals))
    
    tau = 1.0 / lambda_max 

    x = np.zeros_like(b_vec)
    r = A @ x - b_vec
    while np.linalg.norm(r) > eps:
        x = x - tau * r
        r = A @ x - b_vec
        # print(f"norm(r) = {np.linalg.norm(r)}\n")

    return x

In [ ]:
u_rich = Richardson(A, b_vec)